In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Imports section
import sys
sys.path.append('/content/drive/MyDrive/StackGAN/Modules')

import stackgan_stage1 as stg1
import generate_dataset as gd # Self made dataset pipelining module
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import os
import pickle
import time
import cv2

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1024)]               0         []                            
                                                                                                  
 dense (Dense)               (None, 256)                  262400    ['input_1[0][0]']             
                                                                                                  
 leaky_re_lu (LeakyReLU)     (None, 256)                  0         ['dense[0][0]']               
                                                                                                  
 lambda (Lambda)             (None, 128)                  0         ['leaky_re_lu[0][0]']         
                                                                                              

**Conditional Augmentation**

In [ ]:
# Define the conditional augmentation function
def conditional_augmentation(x):
  mean=x[:, :128]
  log_sigma=x[:, 128:]
  stddev = tf.exp(log_sigma)
  epsilon = tf.random.normal(tf.shape(mean), dtype=tf.float32)
  c = mean + stddev * epsilon
  return c

# Build the Conditional Augmentation Network (CAN) model
def build_ca_network(input_shape=(1024,), latent_dim=256):
  input_layer = tf.keras.layers.Input(shape=input_shape)
  x = tf.keras.layers.Dense(latent_dim)(input_layer)
  x = tf.keras.layers.LeakyReLU(x)
  ca = tf.keras.layers.Lambda(conditional_augmentation)(x)

  # Compiling the model
  model = tf.keras.models.Model(inputs=input_layer, outputs=ca)
  model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy')

  return model

# Build an embedding compressor to 128 for Char_CNN_RNN
def embedding_compressor():
  input=tf.keras.layers.Input(shape=(1024,))
  compressed=tf.keras.layers.Dense(units=128)(input)
  compressed=tf.keras.layers.LeakyReLU(alpha=0.01)(compressed)

  model=tf.keras.Model(inputs=[input], outputs=[compressed])
  return model

**Kullback Liebler Divergence Loss**

In [ ]:
# KL Divergence is used as a regularization and latent space control parameter
def KL_adversarial_loss(y_true, y_pred):
  mean=y_pred[:, :128]
  logsigma=y_pred[:, 128:]
  return K.mean(-logsigma+0.5*(-1+K.exp(2.0*logsigma)+K.square(mean)))

**Miscellaneous Functions**

In [ ]:
def save_image(image, filename, img_path):
  # Saves image at specified file path
  if os.path.exists(img_path)==False:
    os.makedirs(img_path)
  cv2.imwrite((img_path+filename), image)


# Build an embedding compressor to 128 for Char_CNN_RNN
def embedding_compressor():
  input=tf.keras.layers.Input(shape=(1024,))
  compressed=tf.keras.layers.Dense(units=128)(input)
  compressed=tf.keras.layers.LeakyReLU(alpha=0.01)(compressed)

  model=tf.keras.Model(inputs=[input], outputs=[compressed])
  return model

**Stage 2 Generator**

In [ ]:
# Generator Residual Block
def residualGen(input):
  x = tf.keras.layers.Conv2D(512, kernel_size=(3,3), padding='same', use_bias=False, kernel_initializer='he_uniform')(input)
  x = tf.keras.layers.BatchNormalization(gamma_initializer='ones', beta_initializer='zeros')(x)
  x = tf.keras.layers.ReLU()(x)

  x = tf.keras.layers.Conv2D(512, kernel_size=(3,3), padding='same', use_bias=False,kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.BatchNormalization(gamma_initializer='ones', beta_initializer='zeros')(x)

  x = tf.keras.layers.add([x, input])
  x = tf.keras.layers.ReLU()(x)
  return x



def Stage2_Generator():
  input_embedding = tf.keras.Input(shape=(1024,))
  input_images = tf.keras.Input(shape=(64, 64, 3))
  # Conditioning Augmentation
  ca = tf.keras.layers.Dense(256)(input_embedding)
  ca = tf.keras.layers.LeakyReLU(alpha=0.2)(ca)
  c = tf.keras.layers.Lambda(conditional_augmentation)(ca)
  # Downsampling
  x = tf.keras.layers.ZeroPadding2D(padding=(1,1))(input_images)
  x = tf.keras.layers.Conv2D(128, kernel_size=(3,3), strides=1, use_bias=False,kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.ReLU()(x)
  x = tf.keras.layers.ZeroPadding2D(padding=(1,1))(x)
  x = tf.keras.layers.Conv2D(256, kernel_size=(4,4), strides=2, use_bias=False,kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.BatchNormalization(gamma_initializer='ones', beta_initializer='zeros')(x)
  x = tf.keras.layers.ReLU()(x)
  x = tf.keras.layers.ZeroPadding2D(padding=(1,1))(x)
  x = tf.keras.layers.Conv2D(512, kernel_size=(4,4), strides=2, use_bias=False,kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.BatchNormalization(gamma_initializer='ones', beta_initializer='zeros')(x)
  x = tf.keras.layers.ReLU()(x)
  # Concatenate text conditioning block with the encoded image
  concat = concat_custom(c, x)
  # Residual Blocks
  x = tf.keras.layers.ZeroPadding2D(padding=(1,1))(concat)
  x = tf.keras.layers.Conv2D(512, kernel_size=(3,3), use_bias=False, kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.BatchNormalization(gamma_initializer='ones', beta_initializer='zeros')(x)
  x = tf.keras.layers.ReLU()(x)
  x = residualGen(x)
  x = residualGen(x)
  x = residualGen(x)
  x = residualGen(x)
  # Upsampling
  x = UpSample2(x, 512)
  x = UpSample2(x, 256)
  x = UpSample2(x, 128)
  x = UpSample2(x, 64)
  x = tf.keras.layers.Conv2D(3, kernel_size=(3,3), padding='same', use_bias=False, kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.Activation('tanh')(x)
  stage2_gen = tf.keras.Model(inputs=[input_embedding, input_images], outputs=[x])
  return stage2_gen

# Upsample block
def UpSample2(x,num_kernels):
  x = tf.keras.layers.UpSampling2D(size=(2,2))(x)
  x = tf.keras.layers.Conv2D(num_kernels, kernel_size=(3,3), padding='same', strides=1, use_bias=False,kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.BatchNormalization(gamma_initializer='ones', beta_initializer='zeros')(x)
  x = tf.keras.layers.ReLU()(x)
  return x

def concat_custom(c,x): #concatenate text conditioning block with the image
  c = K.expand_dims(c, axis=1)
  c = K.expand_dims(c, axis=1)
  c = K.tile(c, [1, 16, 16, 1])
  return K.concatenate([c, x], axis = 3)

In [ ]:
generator = Stage2_Generator()
generator.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 66, 66, 3)            0         ['input_9[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d_5 (Conv2D)           (None, 64, 64, 128)          3456      ['zero_padding2d[0][0]']      
                                                                                                  
 re_lu (ReLU)                (None, 64, 64, 128)          0         ['conv2d_5[0][0]']      

**Stage 2 Discriminator**

In [ ]:
# Build Stage 2 Discriminator
def Stage2_Discriminator():
  # Input image
  input1 = tf.keras.Input(shape=(256, 256, 3))
  x = tf.keras.layers.Conv2D(64, kernel_size=(4,4), strides=2, padding='same', use_bias=False, kernel_initializer=tf.keras.initializers.HeUniform())(input1)
  x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
  # Downsampling and convoluting image
  x = downSample(x, 128)
  x = downSample(x, 256)
  x = downSample(x, 512)
  x = downSample(x, 1024)
  x = downSample(x, 2048)
  x = downSample(x, 1024, (1,1), 1)
  x = downSample(x, 512, (1,1), 1, False)
  x = disResidualBlock(x)
  x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

  # Concatenating compressing text embedding to generated distribution
  input2 = tf.keras.Input(shape=(4,4,128,)) # Text embedding
  x = tf.keras.layers.concatenate([x, input2])
  x = tf.keras.layers.Conv2D(512, kernel_size=(1,1), padding='same', strides=1, use_bias=False,kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

  # Logit generation
  x= tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.GlorotUniform())(x) # 0 or 1
  x = tf.keras.layers.Activation('tanh')(x)
  # Model
  stage2_dis = tf.keras.Model(inputs=[input1, input2], outputs=[x])
  return stage2_dis

# Downsampling block for Discriminator
def downSample(x, kernels, kernel_size=(4,4), strides=2, activation=True):
  x = tf.keras.layers.Conv2D(kernels, kernel_size=kernel_size, padding='same', strides=strides, use_bias=False, kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.BatchNormalization()(x)

  if activation:
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
  return x

# Discriminator residual block
def disResidualBlock(x):
  xd = downSample(x, 128, (1,1), 1)
  xd = downSample(x, 128, (3,3), 1)
  xd = downSample(x, 512, (3,3), 1, False)
  return tf.keras.layers.add([x,xd])

In [ ]:
discriminator=Stage2_Discriminator()
discriminator.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d_22 (Conv2D)          (None, 128, 128, 64)         3072      ['input_10[0][0]']            
                                                                                                  
 leaky_re_lu_8 (LeakyReLU)   (None, 128, 128, 64)         0         ['conv2d_22[0][0]']           
                                                                                                  
 conv2d_23 (Conv2D)          (None, 64, 64, 128)          131072    ['leaky_re_lu_8[0][0]']       
                                                                                            

**Stage 2 Adversarial Model**

In [ ]:
# Build Stage 2 Adversarial model
def Stage2_Adversarial(generator_model, discriminator_model):
    input1 = tf.keras.Input(shape=(1024,)) # Text embedding
    input2 = tf.keras.Input(shape=(64, 64, 3,)) # Gen1 image
    input3 = tf.keras.Input(shape=(4, 4, 128,)) # Compressed embedding

    img = generator_model([input1, input2]) # Text, Gen1 image
  # Discriminator not trainable during adversarial game
    discriminator_model.trainable = False
  # Model
    discrimOutput = discriminator_model([img, input3])
    adversarial_model = tf.keras.Model(inputs=[input1, input2, input3], outputs=[discrimOutput])
    return adversarial_model

In [ ]:
adversarial=Stage2_Adversarial(generator, discriminator)
adversarial.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 1024)]               0         []                            
                                                                                                  
 input_13 (InputLayer)       [(None, 64, 64, 3)]          0         []                            
                                                                                                  
 model_3 (Functional)        (None, 256, 256, 3)          2864544   ['input_12[0][0]',            
                                                          0          'input_13[0][0]']            
                                                                                                  
 input_14 (InputLayer)       [(None, 4, 4, 128)]          0         []                      

In [ ]:
class Stage2_GAN:
  def __init__(self, train_ds, epochs=400, z_dim=100, batch_size=64, gen_lr=0.001, dis_lr=0.001):
    self.epochs=epochs
    self.z_dim=z_dim
    self.batch_size=batch_size
    self.gen_lr=gen_lr
    self.dis_lr=dis_lr
    self.img_size=256
    # Training dataset
    self.train_ds=train_ds

    # Optimizers to the models
    self.gen_optimizer=tf.keras.optimizers.Adam(beta_1=0.6, beta_2=0.999, learning_rate=self.gen_lr)
    self.dis_optimizer=tf.keras.optimizers.Adam(beta_1=0.6, beta_2=0.999, learning_rate=self.dis_lr)

    # Models
    self.generator=Stage2_Generator()
    self.generator.compile(optimizer=self.gen_optimizer, loss='binary_crossentropy')
    self.discriminator=Stage2_Discriminator()
    self.discriminator.loss_function=tf.nn.sigmoid_cross_entropy_with_logits
    self.discriminator.compile(optimizer=self.dis_optimizer, loss=self.discriminator.loss_function)
    self.stage1_generator=stg1.Stage1_Generator()
    self.stage1_generator.trainable=False
    self.stage1_generator.load_weights('/content/drive/MyDrive/StackGAN/Stage_1/Model_weights/Gen_Epochs:All.h5')

    # Embedding Compressor
    self.embed_compressor=embedding_compressor()
    self.embed_compressor.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy')

    # Adversarial Model
    self.model=Stage2_Adversarial(self.generator, self.discriminator)
    self.model.compile(loss=['binary_crossentropy', KL_adversarial_loss], loss_weights=[1., 2.], optimizer=self.gen_optimizer)

    self.checkpoint2 = tf.train.Checkpoint(gen_optimizer=self.gen_optimizer, discriminator_optimizer=self.dis_optimizer, generator=self.generator,discriminator=self.discriminator)

  def train_GAN(self):
    self.gen_loss_log=[]
    self.dis_loss_log=[]
    start=time.time()
    for i in range(self.epochs):
      gen_epoch_loss=[]
      dis_epoch_loss=[]
      print("<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>")
      print("<><><><><><><><> Started Epoch:",(i+1),"<><><><><><><><><><><><><><><><><><><>")

      # Current batch from dataset
      real_img, real_embed=next(self.train_ds)
      if((i+1)%75==0):
        K.set_value(self.gen_optimizer.learning_rate, self.gen_optimizer.learning_rate/2)
        K.set_value(self.dis_optimizer.learning_rate, self.dis_optimizer.learning_rate/2)

      # Iterating through mini-batches
      num_iters=125
      for iter in range(num_iters):
        if((iter+1)%5==0):
          print(":3 ", end='')

        # To train discriminator on real images-real captions
        if(iter!=0):
          real_img, real_embed=next(self.train_ds)
        real_labels= tf.random.uniform(shape=(self.batch_size, 1), minval = .9, maxval = 1.)

        # To train discriminator on real images-mismatched real captions
        mismatched_img=tf.roll(real_img, shift=1, axis=0)
        mismatched_labels= tf.random.uniform(shape=(self.batch_size, 1), minval = .9, maxval = 1.)

        # To train discriminator on fake images-real captions
        low_fake_img=self.stage1_generator([real_embed, tf.random.normal(shape=(self.batch_size, self.z_dim), stddev=0.2)])
        fake_img=self.generator([real_embed, low_fake_img])
        fake_labels= tf.random.uniform(shape=(self.batch_size, 1), minval = 0., maxval = .1)

        # Real captions compressed and reshaped for discriminator
        compressed_embed=self.embed_compressor.predict_on_batch(real_embed)
        compressed_embed=tf.reshape(compressed_embed, (-1, 1, 1, 128))
        compressed_embed=tf.tile(compressed_embed, (1, 4, 4, 1))

        self.discriminator.trainable=True
        # Train Discriminator manually
        with tf.GradientTape() as dis_tape:
          # Discriminator logits
          real_logits=self.discriminator(inputs=[real_img, compressed_embed])
          fake_logits=self.discriminator(inputs=[fake_img, compressed_embed])
          mismatched_logits=self.discriminator(inputs=[mismatched_img, compressed_embed])

          # Loss for each logit
          loss_real=tf.reduce_mean(self.discriminator.loss_function(real_labels, real_logits))
          loss_fake=tf.reduce_mean(self.discriminator.loss_function(fake_labels, fake_logits))
          loss_mismatched=tf.reduce_mean(self.discriminator.loss_function(mismatched_labels, mismatched_logits))

          # Total discriminator weighted loss
          dis_loss=tf.reduce_mean(0.5*tf.add(loss_real, 0.5*tf.add(loss_fake, loss_mismatched)))

        # Discriminator gradient descent on optimizer
        discriminator_gradient=dis_tape.gradient(dis_loss, self.discriminator.trainable_variables)
        print(discriminator_gradient[-1])
        self.dis_optimizer.apply_gradients(zip(discriminator_gradient, self.discriminator.trainable_variables))
        dis_epoch_loss.append(["Batch:",(iter+1),"|| Loss:", dis_loss])
        self.discriminator.trainable=False

        # Training Adversarial GAN model
        gen_loss=self.model.train_on_batch([real_embed, low_fake_img, compressed_embed], [real_labels])
        gen_epoch_loss.append(["Batch:",(iter+1),"|| Loss:", gen_loss])

      print("\nDiscriminator's Loss after epoch number",i,"is=",dis_loss)
      print("Generator's Loss after epoch number",i,"is=",gen_loss)

      # Saving 20 generator images after every 50 epochs
      if(i%50==0):
        gen_images=self.generator.predict_on_batch([real_embed, tf.random.normal(shape=(self.batch_size, self.z_dim), stddev=0.2)])
        for num, image in enumerate(gen_images[:10]):
          image=127.5*image+127.5
          save_image(image, f'{i+1}_{num}.jpg', '/content/drive/MyDrive/StackGAN/Stage_2/Generated_Images/')

      # Saving model weights after every 50 epochs
      if(i%50==0):
        self.generator.save_weights(f'/content/drive/MyDrive/StackGAN/Stage_2/Model_weights/Gen_Epochs:{i+1}.h5')
        self.discriminator.save_weights(f'/content/drive/MyDrive/StackGAN/Stage_2/Model_weights/Dis_Epochs:{i+1}.h5')

      # Appending losses
      self.gen_loss_log.append(gen_epoch_loss)
      self.dis_loss_log.append(dis_epoch_loss)

      print("<><><><><><><><> Time elapsed:",(time.time()-start),"<><><><><><><><><><><>")
      print("<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>")

    with open('/content/drive/MyDrive/StackGAN/Stage_2/Model_weights/Loss.txt', 'w') as file:
      # Writing Generator Loss Log file
      file.write('Generator Loss:\n')
      for item in self.gen_loss_log:
        item=str(item)+'\n'
        file.write(item)

      # Writing Discriminator Loss Log file
      file.write('Discriminator Loss:\n')
      for item in self.dis_loss_log:
        item=str(item)+'\n'
        file.write(item)

    self.generator.save_weights('/content/drive/MyDrive/StackGAN/Stage_2/Model_weights/Gen_Epochs:All.h5')
    self.discriminator.save_weights('/content/drive/MyDrive/StackGAN/Stage_2/Model_weights/Dis_Epochs:All.h5')

**Train Stage 2 GAN**

In [ ]:
start_time=time.time()

# Getting the train dataset
dataset=gd.Dataset_Generator(img_size=(256, 256))
train=dataset.get_train_dataset()

# Iterator is passed to the model for training
train_iterator=iter(train)

In [ ]:
# Stage 2 GAN model
Stage_2_GAN=Stage2_GAN(train_iterator)
# Training the model
Stage_2_GAN.train_GAN()

print("Total time elapsed in training:", (time.time()-start_time))

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
<><><><><><><><> Started Epoch: 1 <><><><><><><><><><><><><><><><><><><>
tf.Tensor([-0.07821339], shape=(1,), dtype=float32)


In [ ]:
drive.flush_and_unmount()